In [1]:
def CP_Diff_Class(X,Y,xt):
    if len(X) == len(Y): #check if Samples and labels have same length
        labels = list(set(Y)) #Finding out the distinct Labels to assign
        
        predicted_labels = [] #will store the predicted label
        pred_label_conformity_score = []  #stores higher pvalue scores
        other_label_conformity_score = [] #stores lower pvalue scores
        
        for b in range(len(xt)): #Looping through all the test samples
            test = xt[b]  #taking 1 test sample at a time
            
            scores_for_both_classes = [] #stores the pvalues (Here for -1 and +1)
            return_labels = [] #consists of labels (Here -1,+1)
            
            for k in range(len(labels)): #Loop to iterate through labels (Here +1 and -1)
                    x =  X.copy() #Create Copy of X-train
                    y =  Y.copy() #Create Copy of Y-train
                    conformity_scores = [] # to store nearest distance to different class
                    distance_array = [] # to store all distances

                    x.append(test) # Create an Augmented Training Set
                    y.append(labels[k]) #Create Augmented set using a label to consider

                    for i in range(len(x)): #loop to find distances from different classes
                        a1 = x[i][0] #take point 1 from tuple of (point1,point2)
                        b1 = x[i][1] #take point 2 from tuple of (point1,point2)

                        for j in range(len(x)): #looping through all the points
                            if j!=i:  # check its not the same sample
                                if y[i] != y[j]: #check if it has a different label
                                    a2 = x[j][0]
                                    b2 = x[j][1]
                                    t1 = (a1-a2)**2
                                    t2 = (b1-b2)**2
                                    d = np.sqrt(t1+t2) # calculations to find Euclidean Distance
                                    distance_array.append(d) #append each distance to different class in this array
                                    
                        conformity_scores.append(min(distance_array)) #append nearest Distance to array using "min"
                        distance_array.clear() #reset distance array to store next set of distances
                    
                    #After this loop we have all the nearest distances to different class in conformity_scores array
                    
                    test_sample_score = conformity_scores[len(x)-1] #take the test sample's score in a variable
                    conformity_scores.sort(reverse=True) #Sort in Descending Order to calculate rank pessimistically
                    
                    for rank in range(len(conformity_scores)): # loop to find rank
                        if conformity_scores[rank]==test_sample_score:
                            break

                    rank = len(x)-rank #Logic for Pessimistic Approach to find Rank
                    p_value = (rank)/len(x) #formula for p_value
                    
                    scores_for_both_classes.append(p_value) #stores pvalue for all the labels (Here +1 and -1)
                    return_labels.append(labels[k]) #stores corresponding labels

            index_max = np.argmax(scores_for_both_classes) #getting index of higher pvalue
            index_min = np.argmin(scores_for_both_classes) #getting index of lower pvalue
            
            predicted_labels.append(return_labels[index_max])  #stores predicted label
            pred_label_conformity_score.append(scores_for_both_classes[index_max]) #stores higher pvalue
            other_label_conformity_score.append(scores_for_both_classes[index_min]) #stores lower pvalue
            
        return predicted_labels,pred_label_conformity_score,other_label_conformity_score
    else:
        return "Error:Size of Samples and Labels mismatched"

In [2]:
import numpy as np
x = [(0,3),(2,2),(3,3),(-1,1),(-1,-1),(0,1)]
y = [+1,+1,+1,-1,-1,-1]
xt = [(-1,-1),(0,0),(0,1),(2,1),(2,3),(3,3)]
# CP_Diff_Class(x,y,xt)

In [3]:
def CP_DiffBySame(X,Y,xt):
    if len(X) == len(Y): #check if Samples and labels have same length
        labels = list(set(Y)) #Finding out the distinct Labels to assign
        
        predicted_labels = [] #will store the predicted label
        pred_label_conformity_score = []  #stores higher pvalue scores
        other_label_conformity_score = [] #stores lower pvalue scores

        for b in range(len(xt)): #Looping through all the test samples
            test = xt[b]  #taking 1 test sample at a time
            
            scores_for_both_classes = [] #stores the pvalues (Here for -1 and +1)
            return_labels = [] #consists of labels (Here -1,+1)
            
            for k in range(len(labels)): #Loop to iterate through labels (Here +1 and -1)
                    x =  X.copy() #Create Copy of X-train
                    y =  Y.copy() #Create Copy of Y-train
                    
                    diff_class = [] #to store nearest distances to different classes
                    same_class = [] #to store nearest distances to same classes
                    
                    conformity_scores = [] #to store diff/same class calculations

                    x.append(test) # Create an Augmented Training Set
                    y.append(labels[k]) #Create Augmented set using a label to consider

                    for i in range(len(x)): #loop to find distances from different classes
                        a1 = x[i][0] #take point 1 from tuple of (point1,point2)
                        b1 = x[i][1] #take point 2 from tuple of (point1,point2)
                        distance_array1 = [] #to store all distances to different class
                        distance_array2 = [] #to store all distances to different class
                        for j in range(len(x)): #looping through all the points
                            if j!=i: #check its not the same point
                                a2 = x[j][0]
                                b2 = x[j][1]
                                t1 = (a1-a2)**2
                                t2 = (b1-b2)**2
                                d = np.sqrt(t1+t2) # formula to find Euclidean Distances   
                                if y[i] != y[j]: #check if it is different class
                                    distance_array1.append(d) #storing Different Classes Distances
                                else: #else it is same class
                                    distance_array2.append(d) #storing Same Classes Distances

                        diff_class.append(min(distance_array1)) #stores Nearest Distance to Different Class
                        same_class.append(min(distance_array2)) #stores Nearest Distance to Same Class

                    for r in range(len(same_class)):  #remove zero values
                        if same_class[r] == 0:
                            same_class[r] = 0.00001
                            
                    #Logic to calculate find Diff/Same Scores
                    conformity_scores = [ p/q for p,q in zip(diff_class,same_class) ]   

                    test_sample_score = conformity_scores[len(x)-1]
                    conformity_scores.sort(reverse=True) #Sort in Descending Order to Calculate rank Pessimistically
                    
                    for rank in range(len(x)): #Loop to find the rank
                        if conformity_scores[rank]==test_sample_score:
                            break

                    rank = len(x)-rank #Logic for Pessimistic Approach to find Rank
                    p_value = (rank)/len(x) #formula for p_value
                    
                    scores_for_both_classes.append(p_value) #stores pvalue for all the labels (Here +1 and -1)
                    return_labels.append(labels[k]) #stores corresponding labels

            index_max = np.argmax(scores_for_both_classes) #getting index of higher pvalue
            index_min = np.argmin(scores_for_both_classes) #getting index of lower pvalue
            
            predicted_labels.append(return_labels[index_max])  #stores predicted label
            pred_label_conformity_score.append(scores_for_both_classes[index_max]) #stores higher pvalue
            other_label_conformity_score.append(scores_for_both_classes[index_min]) #stores lower pvalue
 
        return predicted_labels,pred_label_conformity_score,other_label_conformity_score
    else:
        return "Error:Size of Samples and Labels mismatched"
    

In [4]:
import numpy as np
x = [(0,3),(2,2),(3,3),(-1,1),(-1,-1),(0,1)]
y = [+1,+1,+1,-1,-1,-1]
xt = [(-1,-1),(0,0),(0,2),(2,1),(2,3),(3,3),(0,-2),(-2,0)]
# CP_DiffBySame(x,y,xt)

# Program to Create Artificial Data

In [5]:
import numpy as np
import random

random.seed(3)

x_train_positive=[]
x_m = 2
y_m = 3
s = 1 

for i in range(60):
        x1 = x_m + s*np.random.normal(0,1)
        x2 = y_m + s*np.random.normal(0,1)
        x_train_positive.append((x1,x2))

x_train_negative=[]
x_m = 0
y_m = 0
s = 2
for i in range(70):
        x1 = x_m + s*np.random.normal(0,1)
        x2 = y_m + s*np.random.normal(0,1)
        x_train_negative.append((x1,x2))
    
x_test_positive=[]
x_m = 2
y_m = 3
s = 1 
for i in range(70):
        x1 = x_m + s*np.random.normal(0,1)
        x2 = y_m + s*np.random.normal(0,1)
        x_test_positive.append((x1,x2))

x_test_negative=[]
x_m = 0
y_m = 0
s = 2
for i in range(60):
        x1 = x_m + s*np.random.normal(0,1)
        x2 = y_m + s*np.random.normal(0,1)
        x_test_negative.append((x1,x2))
        

import random
x_train = x_train_positive + x_train_negative
y_train = len(x_train_positive)*[+1] + len(x_train_negative)*[-1]
t = list(zip(x_train, y_train))
random.shuffle(t)
x_train, y_train = zip(*t)
#Here our x_train and y_train Data is ready

x_test = x_test_positive + x_test_negative
y_test = len(x_test_positive)*[+1] + len(x_test_negative)*[-1]
t = list(zip(x_test, y_test))
random.shuffle(t)
x_test, y_test = zip(*t)
#Here our x_test and y_test Data is ready

x_train = list(x_train)
y_train = list(y_train)
x_test = list(x_test)
y_test = list(y_test)
        

# Accuracy Test 

In [6]:
def Accuracy(y_pred,y_test):
    count = 0
    for i in range(len(y_pred)):
        if(y_pred[i]==y_test[i]):
            count+=1
    return (count/len(y_pred))*100

out1 = CP_Diff_Class(x_train,y_train,x_test)
y_pred = out1[0]
A = Accuracy(y_pred,y_test)
print("Accuracy using Different Class is: ",A)

out2 = CP_DiffBySame(x_train,y_train,x_test)
y_pred = out2[0]
A = Accuracy(y_pred,y_test)
print("Accuracy using Diff/Same Class is: ",A)

Accuracy using Different Class is:  83.84615384615385
Accuracy using Diff/Same Class is:  84.61538461538461


# Validity Check

In [7]:
import math
out1 = CP_Diff_Class(x_train,y_train,x_test)
y_pred = out1[0]
p_values = out1[1]
other_pvalues = out1[2]
p_true = [] #True label of Y's Pvalue
for i in range(len(y_pred)):
    if y_test[i] == y_pred[i]:
        p_true.append(p_values[i])   
    else:
        p_true.append(other_pvalues[i])

v = sum(p_true)/len(p_true)
print("Validity for Different Class method is: ",v)

Validity for Different Class method is:  0.4846153846153844


In [8]:
out2 = CP_DiffBySame(x_train,y_train,x_test)
y_pred = out2[0]
p_values = out2[1]
other_pvalues = out1[2]
p_true = [] #True label of Y's Pvalue
for i in range(len(y_pred)):
    if y_test[i] == y_pred[i]:
        p_true.append(p_values[i])
    else:
        p_true.append(other_pvalues[i])

v = sum(p_true)/len(p_true)
print("Validity for Diff/Same Class method is: ",v)

Validity for Diff/Same Class method is:  0.5120963006459189


# Efficiency Test 

In [9]:
out1 = CP_Diff_Class(x_train,y_train,x_test)
pvalues = out1[1] 
other_pvalues = out1[2]
s = []
for (a,b) in zip(pvalues,other_pvalues):
    s.append(a+b)
sum_pvalues = sum(s)
length = len(pvalues)
Efficiency = sum_pvalues/length
print("Efficiency for Different Class method is: ",Efficiency)

out2 = CP_DiffBySame(x_train,y_train,x_test)
pvalues = out2[1] 
other_pvalues = out2[2]
s = []
for (a,b) in zip(pvalues,other_pvalues):
    s.append(a+b)
sum_pvalues = sum(s)
length = len(pvalues)
Efficiency = sum_pvalues/length
print("Efficiency for Diff/Same Class method is: ",Efficiency)

Efficiency for Different Class method is:  0.637874339401057
Efficiency for Diff/Same Class method is:  0.5918966529653554


#  Set Seed Value testing

In [10]:
import numpy as np
import random
import math

set_seeds = []
Acc_1 = []
Acc_2 = []
Val_1 = []
Val_2 = []
Eff_1 = []
Eff_2 = []

for it in range(10):
    random.seed(it)
    set_seeds.append(it)
    x_train_positive=[]
    x_m = 2
    y_m = 3
    s = 1 

    for i in range(60):
            x1 = x_m + s*np.random.normal(0,1)
            x2 = y_m + s*np.random.normal(0,1)
            x_train_positive.append((x1,x2))

    x_train_negative=[]
    x_m = 0
    y_m = 0
    s = 2
    for i in range(70):
            x1 = x_m + s*np.random.normal(0,1)
            x2 = y_m + s*np.random.normal(0,1)
            x_train_negative.append((x1,x2))

    x_test_positive=[]
    x_m = 2
    y_m = 3
    s = 1 
    for i in range(70):
            x1 = x_m + s*np.random.normal(0,1)
            x2 = y_m + s*np.random.normal(0,1)
            x_test_positive.append((x1,x2))

    x_test_negative=[]
    x_m = 0
    y_m = 0
    s = 2
    for i in range(60):
            x1 = x_m + s*np.random.normal(0,1)
            x2 = y_m + s*np.random.normal(0,1)
            x_test_negative.append((x1,x2))


    import random
    x_train = x_train_positive + x_train_negative
    y_train = len(x_train_positive)*[+1] + len(x_train_negative)*[-1]
    t = list(zip(x_train, y_train))
    random.shuffle(t)
    x_train, y_train = zip(*t)
    #Here our x_train and y_train Data is ready

    x_test = x_test_positive + x_test_negative
    y_test = len(x_test_positive)*[+1] + len(x_test_negative)*[-1]
    t = list(zip(x_test, y_test))
    random.shuffle(t)
    x_test, y_test = zip(*t)
    #Here our x_test and y_test Data is ready

    x_train = list(x_train)
    y_train = list(y_train)
    x_test = list(x_test)
    y_test = list(y_test)
    
    out1 = CP_Diff_Class(x_train,y_train,x_test)
    y_pred = out1[0]
    A1 = Accuracy(y_pred,y_test)
    Acc_1.append(A1)
    
    out2 = CP_DiffBySame(x_train,y_train,x_test)
    y_pred = out2[0]
    A2 = Accuracy(y_pred,y_test)
    Acc_2.append(A2)

    out1 = CP_Diff_Class(x_train,y_train,x_test)
    y_pred = out1[0]
    p_values = out1[1]
    other_pvalues = out1[2]
    p_true = [] #True label of Y's Pvalue
    for i in range(len(y_pred)):
        if y_test[i] == y_pred[i]:
            p_true.append(p_values[i])   
        else:
            p_true.append(other_pvalues[i])

    v1 = sum(p_true)/len(p_true)
    Val_1.append(v1)
    
    out2 = CP_DiffBySame(x_train,y_train,x_test)
    y_pred = out2[0]
    p_values = out2[1]
    other_pvalues = out1[2]
    p_true = [] #True label of Y's Pvalue
    for i in range(len(y_pred)):
        if y_test[i] == y_pred[i]:
            p_true.append(p_values[i])
        else:
            p_true.append(other_pvalues[i])

    v2 = sum(p_true)/len(p_true)
    Val_2.append(v2)
    
    out1 = CP_Diff_Class(x_train,y_train,x_test)
    pvalues = out1[1] 
    other_pvalues = out1[2]
    s = []
    for (a,b) in zip(pvalues,other_pvalues):
        s.append(a+b)
    sum_pvalues = sum(s)
    length = len(pvalues)
    E1 = sum_pvalues/length
    Eff_1.append(E1)
    
    out2 = CP_DiffBySame(x_train,y_train,x_test)
    pvalues = out2[1] 
    other_pvalues = out2[2]
    s = []
    for (a,b) in zip(pvalues,other_pvalues):
        s.append(a+b)
    sum_pvalues = sum(s)
    length = len(pvalues)
    E2 = sum_pvalues/length
    Eff_2.append(E2)

In [19]:
from tabulate import tabulate
print("\nObservation Table:")
print("1 stands for Method 1: Nearest Neighbours to Different Class")
print("2 stands for Method 2: Nearest Neighbours to Diff/Same Class\n")

print(tabulate({'Seed Values': set_seeds, 'Accuracy1': Acc_1,
                'Accuracy2': Acc_2, 'Validity1': Val_1, 'Validity2': Val_2,
               'Efficiency1': Eff_1, 'Efficiency2': Eff_2}, headers="keys", tablefmt='fancy_grid', numalign='center'))


Observation Table:
1 stands for Method 1: Nearest Neighbours to Different Class
2 stands for Method 2: Nearest Neighbours to Diff/Same Class

╒═══════════════╤═════════════╤═════════════╤═════════════╤═════════════╤═══════════════╤═══════════════╕
│  Seed Values  │  Accuracy1  │  Accuracy2  │  Validity1  │  Validity2  │  Efficiency1  │  Efficiency2  │
╞═══════════════╪═════════════╪═════════════╪═════════════╪═════════════╪═══════════════╪═══════════════╡
│       0       │   83.8462   │   84.6154   │  0.504228   │   0.54081   │   0.749912    │   0.653435    │
├───────────────┼─────────────┼─────────────┼─────────────┼─────────────┼───────────────┼───────────────┤
│       1       │   84.6154   │   84.6154   │  0.453318   │  0.444862   │   0.659542    │    0.52613    │
├───────────────┼─────────────┼─────────────┼─────────────┼─────────────┼───────────────┼───────────────┤
│       2       │   83.0769   │   85.3846   │  0.459014   │  0.464709   │   0.628303    │   0.523429    │
├────────